In [10]:
import re

In [11]:
file_path = "Recipes/recipes_data.txt"

In [12]:
class Recipes:
    def __init__(self, recipes_file_name, output_file_path = "Recipes/", serial_title = "Combinations", column_separator = "|", separator = "-",
                 text_position = "center", left_padding = 0, right_padding = 0, column_width = 0):
        self.recipes_file_name = recipes_file_name
        self.output_file_path = output_file_path
        self.serial_title = serial_title
        self.column_separator = column_separator
        self.separator = separator
        self.text_position = text_position
        self.left_padding = left_padding
        self.right_padding = right_padding
        self.column_width = column_width


    def get_recipe_names(self):
        """
        This function returns a
        list of names of the dishes.
        """
        
        with open(self.recipes_file_name, 'r') as f:
             content = f.read()
        
        self.content = content
        names = []
        for name in re.findall(r"\w+\s*\w+:", content):
            name = re.sub(":", "", name)
            names.append(name)

        self.names = names
        return  self.names



    def get_ingredients(self):
        """
        This function returns a dictionary with
        names of dishes as keys and a subdictionary
        with its keys as ingredient names and its
        value as a list of possible weights.
        """
        
        ingredients = {}
        for name in self.names:
            ingredients[name] = {}
            pattern = re.compile(f"{name}:\n[-]+\n(.*?)\n[-]+", re.DOTALL)
            for text in re.findall(pattern, self.content):
                for string in re.findall(r"^(?!-).*$", text, re.MULTILINE):
                    for ingredient in re.findall(r"(.*?)\s*=", string, re.DOTALL):
                        ingredients[name][ingredient] = re.findall(r"\d+%*.*?\s*\w*.*?", string, re.DOTALL)

        self.ingredients = ingredients
        return self.ingredients



    def get_recipes(self):
        serial_title = self.serial_title
        
        path = self.output_file_path
        for name in self.names:
            file_name = path + name + str("_recipes.txt")
            ingredients = self.ingredients[name]
            with open(file_name, 'w') as f:
                headers = [serial_title]
                for ingredient_names in ingredients.keys():
                    headers.append(ingredient_names)
                
                title = name + " Recipes."
                f.write(f"{title: ^{len(title)}}" + "\n\n")
                self.write_combinations(f, headers, ingredients)



                
    def get_combinations(self, ingredients, number_of_cells):
        """
        This function returns the different
        possible combinations of ingredients
        for each recipe.
        """
        keys = list(ingredients.keys())

        options = [ingredients[k] for k in keys]
        k = len(keys)
        indices = [0] * k
        combinations = []

        while True:
            for i in range(k):
                ingredient = keys[i]
                option_index = indices[i]
                combinations.append(options[i][option_index])


            j = k - 1
            while j >= 0:
                indices[j] += 1
                if indices[j] == len(options[j]):
                    indices[j] = 0
                    j -= 1
                    
                else:
                    break

            if j < 0:
                break
        
        # Breaking a combinations list into chunks of size n.
        n = number_of_cells - 1
        combinations = [combinations[i * n:(i + 1) * n] for i in range((len(combinations) + n - 1) // n )]

        return combinations




    def write_combinations(self, file, headers, ingredients):
        """
        This function writes the different
        possible combinations of ingredients
        for each recipe.
        """
        
        column_separator = self.column_separator
        separator = self.separator
        text_position = self.text_position
        left_padding = self.left_padding
        right_padding = self.right_padding
        column_width = self.column_width

        number_of_cells = len(headers)
        
        combinations = self.get_combinations(ingredients, len(headers))
        
        for n, combination in enumerate(combinations, start = 1):
            combination.insert(0, str(n))
        
        l = left_padding
        r = right_padding
        max_cell_lengths = [max([len(h), len(c)]) + l + r for h, c in zip(headers, combinations[:][-1])]
        
        for max_cell_length in max_cell_lengths:
            if column_width != 0 and column_width < max_cell_length:
                raise Exception("Column width should be greater than or equal to the maximum cell")

        
        # Calculating the number of rows (or number of possible combinations).
        N = 1
        for key in list(ingredients.keys()):
            N *= len(ingredients[key])
            
        combination_text = ""
        
        for combination in combinations:
            for i in range(len(combination)):
                string = combination[i]
                
                max_cell_length = column_width if column_width != 0 else max_cell_lengths[i]
                spacing = max_cell_length - len(string)
                
                if i == 0:
                    # Filling the first cell.
                    combination_text += column_separator + self.get_aligned_text(string, spacing) + column_separator
                else:
                    combination_text += self.get_aligned_text(string, spacing) + column_separator

            # Condition for adding a newline untill the last entry.
            if int(combination[0]) < N:
                combination_text += "\n"
                
                
        file.write(self.get_separators(max_cell_lengths, number_of_cells, add_column_separation = False))
        file.write("\n")
        
        for a in range(len(headers)):
            header = headers[a]
            
            # Number of white spaces in a cell.
            max_cell_length = column_width if column_width != 0 else max_cell_lengths[a]
            spacing = max_cell_length - len(header)
            
            # print(f"{header}, {len(header)}, {max_cell_length}, {spacing}")
            
            if a == 0:
                # Filling the first cell.
                title = column_separator + self.get_aligned_text(header, spacing) + column_separator
                file.write(title)
                
            else:
                title = self.get_aligned_text(header, spacing) + column_separator
                file.write(title)
        
        file.write("\n")
        file.write(self.get_separators(max_cell_lengths, number_of_cells))
        file.write("\n")
        
        file.write(combination_text)
        
        # Writing ending line of table.
        file.write("\n")
        file.write(self.get_separators(max_cell_lengths, number_of_cells, add_column_separation = False))
        
        
        
    
    def try_recipe(self):
        recipe_name = input("Enter recipe name: ")
        
        if recipe_name not in self.names:
            raise Exception("Recipe not found.")
        
        path = self.output_file_path
        file_name = path + recipe_name + str("_recipes.txt")
        
        with open(file_name, 'r') as f:
            lines = f.readlines()
        
        scorecard = [[" ", " ", " ", " "]] * (len(lines) - 6)
        index = 5
        while index < len(lines):
            line = lines[index]
            quantities = line.strip("|").split("|")
            quantities = [re.sub("(^\s*|\s*$)", "", a) for a in quantities]
            
            quantities.insert(0, "Combination")
            quantities.insert(2, ":")
            quantities = " ".join(quantities)
            print(quantities)
            
            taste = int(input("Taste of this combination (/15): "))
            if taste < 0 or taste > 15:
                raise ValueError("Taste should be between 0 and 15.")
            
            texture = int(input("Texture of this combination (/10): "))
            if texture < 0 or texture > 10:
                raise ValueError("Texture should be between 0 and 10.")
            
            looks = int(input("Looks of this combination (/5): "))
            if looks < 0 or looks > 5:
                raise ValueError("Looks should be between 0 and 5.")
            
            
            total = taste + texture + looks

            print("Total rating (/30): ", total)
            print("\n")

            taste = str(taste) + "/15"
            texture = str(texture) + "/10"
            looks = str(looks) + "/5"
            total = str(total) + "/30"
            # scorecard.append([taste, texture, looks, total])
            scorecard[index - 5] = [taste, texture, looks, total]

            enter = input("Press enter key to continue.")

            if enter == "":
                index += 1
            else:
                break
            
            print("\n")

        headers = ["Taste", "Texture", "Looks", "Total Rating"]

        column_separator = self.column_separator
        l = self.left_padding
        r = self.right_padding
        column_width = self.column_width
        max_cell_lengths = [max([len(h), len(c)]) + l + r for h, c in zip(headers, scorecard[:][-1])]
        
        for max_cell_length in max_cell_lengths:
            if column_width != 0 and column_width < max_cell_length:
                raise Exception("Column width should be greater than or equal to the maximum cell")

        # scorecard.insert(0, headers)
        print(scorecard)
        
        scorecard_file = path + str("chef_scorecard.txt")
        with open(scorecard_file, 'w') as f:
            for line in lines[:2]:
                f.write(line)
            
            f.write(re.sub("\n", "", lines[2]))
            f.write(self.get_separators(max_cell_lengths, len(headers), add_column_separation = False)[1:])
            f.write("\n")
            f.write(re.sub("\n", "", lines[3]))
            
            cursor_position = f.tell()
            f.seek(cursor_position)
            
            for a in range(len(headers)):
                header = headers[a]

                # Number of white spaces in a cell.
                max_cell_length = column_width if column_width != 0 else max_cell_lengths[a]
                spacing = max_cell_length - len(header)

                title = self.get_aligned_text(header, spacing) + column_separator
                f.write(title)
            
            f.write("\n")
            f.write(re.sub("\n", "", lines[4]))
            
            cursor_position = f.tell()
            f.seek(cursor_position)

            f.write(self.get_separators(max_cell_lengths, len(headers))[1:])
            f.write("\n")

            for line, score  in zip(lines[5:], scorecard):
                line = re.sub("\n", "", line)
                f.write(line)

                cursor_position = f.tell()
                f.seek(cursor_position)
                extension = ""
                for i in range(len(score)):
                    string = score[i]
                    max_cell_length = column_width if column_width != 0 else max_cell_lengths[i]
                    spacing = max_cell_length - len(string)
                    extension += self.get_aligned_text(string, spacing) + column_separator
                    
                f.write(extension)
                # del scorecard[scorecard.index(score)]
                f.write("\n")
            
            f.write(re.sub("\n", "", lines[2]))
            f.write(self.get_separators(max_cell_lengths, len(headers), add_column_separation = False)[1:])
    
    
    
    
    def get_aligned_text(self, text, spacing):
        """
        This function sets the alignment
        of the text and returns aligned text.
        """

        text_position = self.text_position
        l = self.left_padding
        r = self.right_padding
        
        if text_position == "left":
            spacing = spacing - l - r if l or r != 0 else spacing
            d = spacing + len(text)
            aligned_text = " " * l + f"{text : <{d}}" + " " * r
            
        elif text_position == "center":
            spacing = spacing - l - r if l or r != 0 else spacing
            d = spacing + len(text)
            aligned_text = " " * l + f"{text : ^{d}}" + " " * r
            
        elif text_position == "right":        
            spacing = spacing - l - r if l or r != 0 else spacing
            d = spacing + len(text)
            aligned_text = " " * l + f"{text : >{d}}" + " " * r
            
        else:
            raise ValueError("Enter the correct alignment. The correct alignments are left, center and right.")

        return aligned_text




    def get_separators(self, max_cell_lengths, number_of_cells, add_column_separation = True):
        """
        This function returns a separator line
        (a separation between headers and table).
        """

        column_width = self.column_width
        column_separator = self.column_separator
        separator = self.separator
        
        max_cell_lengths = [column_width] * number_of_cells if column_width != 0 else max_cell_lengths
        
        if add_column_separation:
            separator_line = column_separator
            for i in range(number_of_cells):
                separator_line += separator * max_cell_lengths[i] + column_separator
                
        else:
            # Endline.
            separator_line = separator
            for i in range(number_of_cells):
                separator_line += separator * (max_cell_lengths[i] + 1)

        return separator_line

In [13]:
recipes = Recipes(file_path, output_file_path = "Recipes/", serial_title = "Combinations", column_separator = "|", separator = "-",
                  text_position = "center", left_padding = 0, right_padding = 0, column_width = 0)

In [14]:
recipes.get_recipe_names()

['Chocolate', 'Ice Cream', 'Kheer', 'Halwa']

In [15]:
recipes.get_ingredients()

{'Chocolate': {'Sugar': ['2 spoons', '3 spoons', '5 spoons'],
  'Milk': ['1% Fat', '5% Fat', '10% Fat'],
  'Flour': ['5 gm', '10 gm']},
 'Ice Cream': {'Milk': ['500 ml', '1 liter'],
  'Sugar': ['100 gm', '200 gm', '300 gm'],
  'Vanilla Extract': ['1 teaspoon', '2 teaspoons'],
  'Whipping Cream': ['200 ml', '400 ml'],
  'Egg Yolks': ['4', '6']},
 'Kheer': {'Sugar': ['10 spoons', '20 spoons', '30 spoons'],
  'Milk': ['5% Fat', '10% Fat'],
  'Rice': ['500 gm', '600 gm'],
  'Water': ['300 ml', '400 ml']},
 'Halwa': {'Sugar': ['10 gm', '20 gm', '30 gm'],
  'Milk': ['200 ml', '300 ml'],
  'Carrot': ['500 gm', '600 gm', '800 gm']}}

In [16]:
recipes.get_recipes()

In [19]:
recipes.try_recipe()

Enter recipe name: Chocolate
Combination 1 : 2 spoons 1% Fat 5 gm 
Taste of this combination (/15): 12
Texture of this combination (/10): 9
Looks of this combination (/5): 5
Total rating (/30):  26


Press enter key to continue.


Combination 2 : 2 spoons 1% Fat 10 gm 
Taste of this combination (/15): 9
Texture of this combination (/10): 8
Looks of this combination (/5): 5
Total rating (/30):  22


Press enter key to continue.


Combination 3 : 2 spoons 5% Fat 5 gm 
Taste of this combination (/15): 9
Texture of this combination (/10): 5
Looks of this combination (/5): 4
Total rating (/30):  18


Press enter key to continue.


Combination 4 : 2 spoons 5% Fat 10 gm 
Taste of this combination (/15): 6
Texture of this combination (/10): 4
Looks of this combination (/5): 2
Total rating (/30):  12


Press enter key to continue. 
[['12/15', '9/10', '5/5', '26/30'], ['9/15', '8/10', '5/5', '22/30'], ['9/15', '5/10', '4/5', '18/30'], ['6/15', '4/10', '2/5', '12/30'], [' ', ' ', ' ', ' '], [' ',